In [1]:
# I am making it a new tradition for me to check sys.executable to ensure that I am working in the correct environment.
# For the love of god, I can feel more of my bare scalp every week.
import sys
sys.executable

'/home/dave/anaconda3/bin/python'

In [2]:
import json
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
import ollama
from multiprocessing import Pool, cpu_count
import time
import re

In [3]:
data_dir = '/media/dave/Prepper/ol_dump_2025-12-31.txt'

In [4]:
rows = []

In [5]:
#total_lines = sum(1 for _ in open(data_dir, "r", encoding="utf-8"))
#print(f"Total Lines: {total_lines:,}")

In [5]:
total_lines = 116417996

In [6]:
prev_row = None
last_row = None

In [7]:
with open(data_dir, "r", encoding='utf-8') as f:
    for line in tqdm(f, total=total_lines, desc="Scanning dump"):
        try:
            if line.startswith("/type/work"):
                continue

            parts = line.rstrip("\n").split("\t")
            data = json.loads(parts[4])

            if "description" in data: 
                desc = data["description"]
                if isinstance(desc, dict):
                    desc = desc.get("value", "")

                title = data.get("title", None)                
                new_row = (title, desc)

                prev_row = last_row
                last_row = new_row

                rows.append(new_row)

        except Exception as e:
            print("Error:", e)
            print("Previous row:", prev_row)
            print("Last row:", last_row)
            

print(rows[0])

Scanning dump: 100%|██████████| 116417996/116417996 [23:16<00:00, 83340.20it/s] 

('Glass Town', 'A brief, fictionalized account of the daily lives of the Brontës, told from the point of view of Charlotte.')


Extensive effort is being put towards ensuring that every description also has a title. Descriptions with missing titles will either be overlooked, or I will investigate assigning them titles with a lightweight LLM. I consider it important to leave titles in, not because they are important for the research quetsion at hand, but for the sake of scientific clarify and reproducability. I think it is paramount that I limit all ambiguity to the question at hand, with all other factors being known and understood as wholly as is feasible. 

In [25]:
print(type(rows[10]))

<class 'tuple'>


In [26]:
print(rows[0][1])

{'type': '/type/text', 'value': 'A brief, fictionalized account of the daily lives of the Brontës, told from the point of view of Charlotte.'}


In [21]:
print(len(rows))

1961024


In [8]:
clean_rows = []

for title, desc in rows:
    if isinstance(desc, dict):
        desc = desc.get("value", "")
    clean_rows.append((title, desc))

In [9]:
df = pd.DataFrame(clean_rows, columns=["title","description"])
df.head()
df.tail()

,title,description
1955694,None,Q/微信859034112办理毕业证成绩单丶使馆和教育部学历学位认证★诚招代理★\n英华教育...
1955695,None,Books mentioned by Mercedes Shepard English in...
1955696,None,"The noise of time, Theodosia. The Egyptian sta..."
1955697,None,Author databar
1955698,None,"Presents the adventures of Mowgli, a boy reare..."


In [29]:
df.to_csv("clean_rows.csv")

In [45]:
async def get_embeddings(input, embeddings):
    try:
        response = ollama.embeddings(
            model="mxbai-embed-large",
            prompt=input
        )
        embeddings.append(response["embedding"])

    except Exception as e:
        print("Error:", e)
        embeddings.append(None)
        #print("Problematic input of length:", len(input), " was:", input)
        

In [11]:
df_test = df.head(10)
df_test.head(15)

,title,description
0,Glass Town,"A brief, fictionalized account of the daily li..."
1,Coretta Scott King,Explores the life and career of Coretta Scott ...
2,A tawdry place of salvation,In this first critical study of the work of Ja...
3,Inside COM,"""Microsoft's Component Object Model (COM) has ..."
4,Thunderbird gold,"While visiting his cousin in northern Canada, ..."
5,Sanskrit bījas and mantras in Japan,tum kay kar rahio
6,Mount St. Helens National Volcanic Monument,Describes the destruction caused by the erupti...
7,A mood apart,"In A Mood Apart, Dr. Peter Whybrow examines mo..."
8,Between Jihad and Salaam,In over a dozen face-to-face encounters with t...
9,Israel Potter,"""Based on the life of an actual soldier who cl..."


In [12]:
def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+", "", text)   #url's
    text = re.sub(r"[\n\r\t]+", " ", text) #whitespace
    text = re.sub(r"[^ -~]", "", text)    #non-ASCII
    text = re.sub(r"\s{2,}", " ", text)   # spaces
    text = re.sub(r'"', ' ', text)
    return text

In [28]:
embeddings = []
word_counts=[]

for row in tqdm(df_filtered["description"], total=len(df_filtered), desc="Total Rows"):

    clean_text(row)
    #new_embedding = await get_embeddings(str(row))
    word_count = len(str(row).split())
    #embeddings.append(new_embedding)
    word_counts.append(word_count)

print("Total Word Counts:", len(word_counts))
print("Total Embeddings Vectors:", len(embeddings))

Total Rows: 100%|██████████| 1945987/1945987 [00:56<00:00, 34694.73it/s]

Total Word Counts: 1945987
Total Embeddings Vectors: 0


In [29]:
df_filtered['word_count'] = word_counts

In [30]:
df_filtered.head()

,title,description,word_count
0,Glass Town,"A brief, fictionalized account of the daily li...",19
1,Coretta Scott King,Explores the life and career of Coretta Scott ...,31
2,A tawdry place of salvation,In this first critical study of the work of Ja...,133
3,Inside COM,Microsoft's Component Object Model (COM) has ...,146
4,Thunderbird gold,"While visiting his cousin in northern Canada, ...",33


In [22]:
'''
I'd like to finish this process while I'm still young
so the process will be coded to handle multithreading
'''

def filter_chunk(df_chunk):
    df_chunk["description"] = df_chunk["description"].apply(clean_text)
    return df_chunk[df_chunk["word_count"] <= 500]

In [23]:
workers = 4
num_chunks = 200

chunks = np.array_split(df, num_chunks)

with Pool(workers) as p:
    results = list(tqdm(
        p.imap(filter_chunk, chunks),
        total=num_chunks,
        desc="Filtering rows"
    ))

df_filtered = pd.concat(results, ignore_index=True)

/home/dave/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
Filtering rows: 100%|██████████| 200/200 [00:16<00:00, 11.95it/s]


This is, admittedly, a weakness in methodology. I think a better alternative would be to instead create a 512-word summary of works with very long descriptions. It's hard to quantify any potential downstream consequences, but the current methodology seems like it would create a bias towards works than can be concisely described, which could prove to be intellectually shallow if nothing else. 

In [24]:
print(len(df))
print(len(df_filtered))

1955699
1945987


All things considered, a loss of only 10,000 rows isn't too bad

In [127]:
df_filtered.to_csv("shortened_clean_rows.csv")

In [31]:
df_filtered_sample = df_filtered.head(1000)

In [32]:
df_filtered_sample.head()

,title,description,word_count
0,Glass Town,"A brief, fictionalized account of the daily li...",19
1,Coretta Scott King,Explores the life and career of Coretta Scott ...,31
2,A tawdry place of salvation,In this first critical study of the work of Ja...,133
3,Inside COM,Microsoft's Component Object Model (COM) has ...,146
4,Thunderbird gold,"While visiting his cousin in northern Canada, ...",33


In [61]:
df_filtered_sample.to_csv("df_filtered_sample.csv")

In [33]:
print(df_filtered_sample.iloc[483])

title                   The playground of psychoanalytic therapy
description     Taking as its point of departure Freud's noti...
word_count                                                   395
Name: 483, dtype: object


In [46]:
embeddings = []
word_counts=[]

start_time = time.time()
for row in tqdm(df_filtered_sample["description"], total=len(df_filtered_sample), desc="Total Rows"):
    
    #new_embedding = await get_embeddings(str(row))
    await get_embeddings(row, embeddings)
    #word_count = len(str(row).split())
    #embeddings.append(new_embedding)
    #word_counts.append(word_count)

end_time = (time.time() - start_time)
print("Total time:", end_time)
print("Total Word Counts:", len(word_counts))
print("Total Embeddings Vectors:", len(embeddings))

Total Rows:  49%|████▉     | 489/1000 [00:22<00:19, 26.03it/s]

Error: Failed to create new sequence: the input length exceeds the context length (status code: 500)


Total Rows:  82%|████████▏ | 819/1000 [00:36<00:08, 22.12it/s]

Error: Failed to create new sequence: the input length exceeds the context length (status code: 500)


Total Rows:  88%|████████▊ | 882/1000 [00:39<00:05, 20.50it/s]

Error: Failed to create new sequence: the input length exceeds the context length (status code: 500)


Total Rows: 100%|██████████| 1000/1000 [00:44<00:00, 22.72it/s]

Total time: 44.00897669792175
Total Word Counts: 0
Total Embeddings Vectors: 1000


In [50]:
print(len(embeddings))
print(len(df_filtered_sample))

1000
1000


In [56]:
df_filtered_sample['embeddings'] = pd.Series(embeddings, index=df_filtered_sample.index)

/tmp/ipykernel_643537/3365767227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_sample['embeddings'] = pd.Series(embeddings, index=df_filtered_sample.index)


In [57]:
df_filtered_sample.head()

,title,description,word_count,embeddings
0,Glass Town,"A brief, fictionalized account of the daily li...",19,"[0.013128243386745453, -0.638306200504303, 0.1..."
1,Coretta Scott King,Explores the life and career of Coretta Scott ...,31,"[-0.3468349575996399, -0.21785852313041687, -0..."
2,A tawdry place of salvation,In this first critical study of the work of Ja...,133,"[-0.10799460113048553, -1.4149322509765625, -0..."
3,Inside COM,Microsoft's Component Object Model (COM) has ...,146,"[-0.46148014068603516, -0.4759032726287842, 0...."
4,Thunderbird gold,"While visiting his cousin in northern Canada, ...",33,"[0.2797992527484894, 0.21501488983631134, 0.27..."


In [62]:
full_df_sample = pd.DataFrame(df_filtered_sample)

In [65]:
full_df_sample.to_csv("full_df_sample.csv")

In [142]:
df["word_counts"] = word_counts